In [134]:
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import momepy
from shapely.geometry import Point, MultiLineString, LineString
from shapely.ops import nearest_points
import osmnx as ox

In [135]:
ox.

'1.2.2'

In [69]:
road = gpd.read_file('data/link_data.gpkg')

# 각 링크별 길이 계산 (단위: 미터)
road['road_length(km)'] = road.geometry.length/1000

#열 이름 변경
road = road.rename(columns={
    'link_id': 'road_id',
    'sigungu_id': 'road_sigungu_id'
})

#필요한 열만 추출 
road = road[['road_id', 'road_name', 'road_rank','road_sigungu_id','oneway', 'geometry','road_length(km)']]


In [70]:
speeds = pd.read_csv('data/speed_data.csv')
# 열 이름 영어로 변경

In [71]:
speeds = speeds.rename(columns={
    '﻿level6 LINK ID': 'link_id', 
    '평균속도 (km/h)': 'link_avsp',
    '도로등급': 'link_lev',
})
speeds

,Unnamed: 0,link_id,연월,평일 / 주말,주요시간대,link_lev,시도명,시군구명,읍면동명,15% 주행속도 (km/h),25% 주행속도 (km/h),30% 주행속도 (km/h),50% 주행속도 (km/h),75% 주행속도 (km/h),85% 주행속도 (km/h),link_avsp,속도 표준편차 (km/h),최대속도 (km/h)
0,1,45880198001,202210,평일,17,시군도,충청남도,공주시,탄천면,32,32,32,32,35,35,33.50,1.50,35
1,2,45880198201,202210,평일,17,시군도,충청남도,공주시,탄천면,29,29,29,29,31,31,30.00,1.00,31
2,3,45880198202,202210,평일,8,시군도,충청남도,공주시,탄천면,18,18,18,18,21,21,19.50,1.50,21
3,4,45880387201,202210,평일,7,일반국도,충청남도,공주시,탄천면,61,66,66,74,84,98,77.50,14.90,104
4,5,45880387201,202210,평일,8,일반국도,충청남도,공주시,탄천면,58,63,65,70,94,97,78.29,18.63,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142610,142611,92451455502,202210,평일,8,지방도,충청북도,청주시 흥덕구,옥산면,12,21,23,48,75,91,49.62,32.02,106
142611,142612,92451455502,202210,평일,11,지방도,충청북도,청주시 흥덕구,옥산면,3,21,25,66,77,88,52.29,34.02,112
142612,142613,92451455502,202210,평일,12,지방도,충청북도,청주시 흥덕구,옥산면,9,22,28,59,75,82,51.03,31.88,160
142613,142614,92451455502,202210,평일,17,지방도,충청북도,청주시 흥덕구,옥산면,20,27,31,64,73,78,53.66,28.48,122


In [72]:
# 도로등급별 평균속도 계산
road_rank_avg_speed = speeds.groupby('link_lev')['link_avsp'].mean().to_dict()

# link_avsp를 float 타입으로 변환 (null 값이 있는지 확인 전에 타입 변경)
speeds['link_avsp'] = speeds['link_avsp'].astype(float)

# Null 값 처리
def fill_avsp(row, avg_dict):
    if pd.isnull(row['link_avsp']):
        return avg_dict.get(row['link_lev'], np.nan)
    else:
        return row['link_avsp']

speeds['link_avsp'] = speeds.apply(lambda row: fill_avsp(row, road_rank_avg_speed), axis=1)

In [73]:
# 도로 등급에 따른 숫자 지정
link_lev_to_rank = {
    '연결로': 108,
    '시군도': 107,
    '지방도': 106,
    '국가지원지방도': 105,
    '특별광역시도': 104,
    '일반국도': 103,
    '도시고속국도': 102,
    '고속도로': 101,
}
speeds['link_rank'] = speeds['link_lev'].map(link_lev_to_rank)

In [74]:
# 'alink_id'의 앞자리부터 9자리만 사용하여 'modified_link_id' 생성
speeds['modified_link_id'] = speeds['link_id'].apply(lambda x: str(x)[:9])

# modified_link_id와 link_lev를 기준으로 그룹화 후 link_avsp의 평균 계산
speeds_final = speeds.groupby(['modified_link_id', 'link_lev', 'link_rank'])['link_avsp'].mean().reset_index()

In [75]:
# speeds_final과 road를 link_data로 지정
road['road_id'] = road['road_id'].astype(str)
link_data = road.merge(speeds_final, how='left', left_on='road_id', right_on='modified_link_id')
link_data

,road_id,road_name,road_rank,road_sigungu_id,oneway,geometry,road_length(km),modified_link_id,link_lev,link_rank,link_avsp
0,458801980,,107,34020,0,"MULTILINESTRING ((954878.902 1815326.471, 9548...",0.005900,458801980,시군도,107.0,33.500000
1,458801982,,107,34020,0,"MULTILINESTRING ((954881.148 1815321.078, 9548...",0.027669,458801982,시군도,107.0,24.750000
2,458802118,가배실로,107,34020,0,"MULTILINESTRING ((954348.540 1813477.043, 9543...",0.535368,NaN,NaN,NaN,NaN
3,458802119,가배실로,107,34020,0,"MULTILINESTRING ((955132.708 1812933.848, 9549...",0.510799,NaN,NaN,NaN,NaN
4,458803103,가배실로,107,34020,1,"MULTILINESTRING ((954360.539 1813482.446, 9543...",0.027886,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
23363,563417889,가로수로,103,33043,0,"MULTILINESTRING ((983019.752 1845256.029, 9830...",0.011778,563417889,일반국도,103.0,35.367500
23364,563417590,오송가락로,107,33043,0,"MULTILINESTRING ((984067.936 1845988.959, 9840...",0.021421,563417590,시군도,107.0,32.118333
23365,563417594,오송가락로,107,33043,0,"MULTILINESTRING ((984067.936 1845988.959, 9840...",0.020459,563417594,시군도,107.0,26.175000
23366,563417601,,107,33043,1,"MULTILINESTRING ((984093.254 1846009.747, 9840...",0.036607,563417601,시군도,107.0,32.713333


In [80]:
# 'link_rank' 별로 그룹화 후 'link_avsp'의 평균을 계산
rank_ave_speed = link_data.groupby('road_rank')['link_avsp'].mean().to_dict()

# link_avsp의 NaN 값을 해당 road_rank의 평균값으로 기입
link_data['link_avsp'] = link_data.apply(
    lambda row: rank_ave_speed[row['road_rank']] if pd.isnull(row['link_avsp']) else row['link_avsp'],
    axis=1
)

In [87]:
#link_data에 시간을 hour와 min, 분:초 형식으로 시간 변환한 열 추가
link_data['time(m)'] = link_data['road_length(km)']/link_data['link_avsp']*60
link_data['oneway'] = link_data['oneway'].map({1: True, 0: False})

In [105]:
link_data

,road_id,road_name,road_rank,road_sigungu_id,oneway,geometry,road_length(km),modified_link_id,link_lev,link_rank,link_avsp,time(m)
0,458801980,,107,34020,False,"MULTILINESTRING ((954878.902 1815326.471, 9548...",0.005900,458801980,시군도,107.0,33.500000,0.010566
1,458801982,,107,34020,False,"MULTILINESTRING ((954881.148 1815321.078, 9548...",0.027669,458801982,시군도,107.0,24.750000,0.067076
2,458802118,가배실로,107,34020,False,"MULTILINESTRING ((954348.540 1813477.043, 9543...",0.535368,NaN,NaN,NaN,31.094694,1.033041
3,458802119,가배실로,107,34020,False,"MULTILINESTRING ((955132.708 1812933.848, 9549...",0.510799,NaN,NaN,NaN,31.094694,0.985633
4,458803103,가배실로,107,34020,True,"MULTILINESTRING ((954360.539 1813482.446, 9543...",0.027886,NaN,NaN,NaN,31.094694,0.053808
...,...,...,...,...,...,...,...,...,...,...,...,...
23363,563417889,가로수로,103,33043,False,"MULTILINESTRING ((983019.752 1845256.029, 9830...",0.011778,563417889,일반국도,103.0,35.367500,0.019981
23364,563417590,오송가락로,107,33043,False,"MULTILINESTRING ((984067.936 1845988.959, 9840...",0.021421,563417590,시군도,107.0,32.118333,0.040016
23365,563417594,오송가락로,107,33043,False,"MULTILINESTRING ((984067.936 1845988.959, 9840...",0.020459,563417594,시군도,107.0,26.175000,0.046898
23366,563417601,,107,33043,True,"MULTILINESTRING ((984093.254 1846009.747, 9840...",0.036607,563417601,시군도,107.0,32.713333,0.067141


In [106]:
# MultiLineString을 LineString으로 변환
single_link_data = link_data.explode(ignore_index=True)
single_link_data

,road_id,road_name,road_rank,road_sigungu_id,oneway,road_length(km),modified_link_id,link_lev,link_rank,link_avsp,time(m),geometry
0,458801980,,107,34020,False,0.005900,458801980,시군도,107.0,33.500000,0.010566,"LINESTRING (954878.902 1815326.471, 954880.098..."
1,458801982,,107,34020,False,0.027669,458801982,시군도,107.0,24.750000,0.067076,"LINESTRING (954881.148 1815321.078, 954888.033..."
2,458802118,가배실로,107,34020,False,0.535368,NaN,NaN,NaN,31.094694,1.033041,"LINESTRING (954348.540 1813477.043, 954355.020..."
3,458802119,가배실로,107,34020,False,0.510799,NaN,NaN,NaN,31.094694,0.985633,"LINESTRING (955132.708 1812933.848, 954943.343..."
4,458803103,가배실로,107,34020,True,0.027886,NaN,NaN,NaN,31.094694,0.053808,"LINESTRING (954360.539 1813482.446, 954350.624..."
...,...,...,...,...,...,...,...,...,...,...,...,...
23363,563417889,가로수로,103,33043,False,0.011778,563417889,일반국도,103.0,35.367500,0.019981,"LINESTRING (983019.752 1845256.029, 983030.593..."
23364,563417590,오송가락로,107,33043,False,0.021421,563417590,시군도,107.0,32.118333,0.040016,"LINESTRING (984067.936 1845988.959, 984076.826..."
23365,563417594,오송가락로,107,33043,False,0.020459,563417594,시군도,107.0,26.175000,0.046898,"LINESTRING (984067.936 1845988.959, 984079.238..."
23366,563417601,,107,33043,True,0.036607,563417601,시군도,107.0,32.713333,0.067141,"LINESTRING (984093.254 1846009.747, 984085.320..."


In [109]:
G = momepy.gdf_to_nx(single_link_data, approach = 'primal', directed = True, length = 'road_length(km)', oneway_column= 'oneway')

In [96]:
# 도로 네트워크에서 모든 노드(시작점과 끝점)를 추출하는 함수
def extract_nodes(geometry):
    if isinstance(geometry, MultiLineString):
        return [Point(line.coords[0]) for line in geometry.geoms] + \
               [Point(line.coords[-1]) for line in geometry.geoms]
    elif isinstance(geometry, LineString):
        return [Point(geometry.coords[0]), Point(geometry.coords[-1])]
    else:
        raise ValueError(f"Unhandled geometry type: {type(geometry)}")

# link_data로부터 노드를 추출하여 nodes_gdf 생성
all_nodes = []
for _, row in link_data.iterrows():
    all_nodes.extend(extract_nodes(row['geometry']))

nodes_gdf = gpd.GeoDataFrame(geometry=all_nodes)

In [98]:
# nodes_gdf로부터 그래프 생성
G = nx.Graph()
for index, row in link_data.iterrows():
    geom = row['geometry']
    # LineString 객체 처리
    if isinstance(geom, LineString):
        start_point = geom.coords[0]  # 시작점
        end_point = geom.coords[-1]  # 끝점
        G.add_edge(start_point, end_point, weight=row['time(m)'])
    # MultiLineString 객체 처리
    elif isinstance(geom, MultiLineString):
        for line in geom.geoms:
            start_point = line.coords[0]  # 시작점
            end_point = line.coords[-1]  # 끝점
            G.add_edge(start_point, end_point, weight=row['time(m)'])
